<a href="https://colab.research.google.com/github/thaisrezendeb/ufes_machineLearning/blob/main/L2_Exercicio4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte V – Métodos de Classificação Baseados em Procura

**4) Para a base Servo (disponível em http://archive.ics.uci.edu/ml/):**

**a) Construa uma árvore de regressão com dois níveis de nó de decisão (isto é, o primeiro nó de decisão (primeiro nível), os nós de decisão abaixo dele (segundo nível) e em seguida os nós folha) usando a medida de redução de desvio padrão. Selecione aleatoriamente 75% dos dados para treinamento que serão usados para construir a árvore. Retorne a estrutura da árvore construída.**



In [ ]:
import pandas as pd
import numpy as np

columns_servo = ["motor", "screw", "pgain", "vgain", "class"]
uri = "https://archive.ics.uci.edu/ml/machine-learning-databases/servo/servo.data"
servo = pd.read_csv(uri, header=None)#, names=columns_servo)

#Conversões necessárias para usar o algoritmo
servo.replace('A', 0, inplace=True)
servo.replace('B', 1, inplace=True)
servo.replace('C', 2, inplace=True)
servo.replace('D', 3, inplace=True)
servo.replace('E', 4, inplace=True)

#Seleção aleatória das amostras
np.random.seed(7)
size_train = 0.75
sel_train = np.random.choice(servo.shape[0], round(servo.shape[0]*size_train), replace=False)
servo_train = servo.iloc[sel_train]
servo_test = servo.drop(sel_train)

def sd_calc(input):
  sd_calc = np.sqrt(
              np.divide(
                np.sum(
                  np.power(input - input.mean(), 2)) \
                , input.shape[0]))
            
  return sd_calc

def better_div(df: pd.DataFrame):
  #Desvio padrão da variável alvo
  sd_class = sd_calc(df[df.columns.values[-1]])

  #Limites para divisão das amostras
  limites = np.zeros((df.shape[1]))

  #Seleção da melhor divisão para cada atributo
  for i in df.columns:
    if i == 4:
      continue

    max_sdr = -999
    for j in df[i].unique():
      servo_train_r = df[df[i] <= j]
      servo_train_s = df.drop(servo_train_r.index)

      sdr_r = sdr_s = 0
      if servo_train_r.shape[0] > 0:
        sdr_r = servo_train_r.shape[0]/df.shape[0] \
                  * sd_calc(servo_train_r[df.columns.values[-1]])

      if servo_train_s.shape[0] > 0:
        sdr_s = servo_train_s.shape[0]/df.shape[0] \
                  * sd_calc(servo_train_s[df.columns.values[-1]])
      
      sdr = sd_class - sdr_r - sdr_s
      
      if sdr > max_sdr:
        max_sdr = sdr
        limites[i] = j

    print("SDR(class, {} <= {}) = {:.4f}".format(columns_servo[i], limites[i], max_sdr))

  return 

In [ ]:
#Attribute Information:
#1. motor: A,B,C,D,E
#2. screw: A,B,C,D,E
#3. pgain: 3,4,5,6
#4. vgain: 1,2,3,4,5
#5. class: 0.13 to 7.10

better_div(servo_train)

SDR(class, motor <= 1.0) = 0.0489
SDR(class, screw <= 0.0) = 0.0581
SDR(class, pgain <= 3.0) = 0.8647
SDR(class, vgain <= 2.0) = 0.3659


Baseados nos cálculos de redução do desvio padrão, o nó de decisão do primeiro nível é "pgain" com divisão Nr = {3} e Ns = {4,5,6}.

In [ ]:
#Definição dos nós do segundo nível

#Para a aresta R
servo_r = servo_train[servo_train[2] <= 3]
servo_r = servo_r.drop(2, axis=1)
better_div(servo_r)

SDR(class, motor <= 1.0) = 0.4947
SDR(class, screw <= 0.0) = 0.2918
SDR(class, vgain <= 1.0) = 0.0379


Para o nó da aresta R será considerado o atributo "motor" com divisão Nr = {A,B} e Ns = {C,D,E}

In [ ]:
#Para a aresta S
servo_s = servo_train.drop(servo_r.index)
servo_s = servo_s.drop(2, axis=1)
better_div(servo_s)

SDR(class, motor <= 3.0) = 0.0010
SDR(class, screw <= 1.0) = 0.0212
SDR(class, vgain <= 3.0) = 0.0207


O nó da aresta S será o atributo "screw" com divisão Nr = {A,B} e Ns = {C,D,E}. Portanto, a árvore de regressão será:

In [ ]:
#Médias para folhas

folha1 = servo_train[(servo_train[2] <= 3) & (servo_train[0] <= 1)]
folha1_val = folha1[4].mean()
print("Folha 1 -> # amostras: {} - média: {:.4f}".format(folha1.shape[0], folha1_val))

folha2 = servo_train[(servo_train[2] <= 3) & (servo_train[0] > 1)]
folha2_val = folha2[4].mean()
print("Folha 2 -> # amostras: {} - média: {:.4f}".format(folha2.shape[0], folha2_val))

folha3 = servo_train[(servo_train[2] > 3) & (servo_train[1] <= 1)]
folha3_val = folha3[4].mean()
print("Folha 3 -> # amostras: {} - média: {:.4f}".format(folha3.shape[0], folha3_val))

folha4 = servo_train[(servo_train[2] > 3) & (servo_train[1] > 1)]
folha4_val = folha4[4].mean()
print("Folha 4 -> # amostras: {} - média: {:.4f}".format(folha4.shape[0], folha4_val))


Folha 1 -> # amostras: 17 - média: 4.5823
Folha 2 -> # amostras: 25 - média: 2.3960
Folha 3 -> # amostras: 39 - média: 0.7284
Folha 4 -> # amostras: 44 - média: 0.5250


A árvore final é:
<figure>
<center>
<img src='https://drive.google.com/uc?id=1rT71wQvD1B9znPJyZOaWbHlGp7nIw7pD' />
</center>
</figure>


**b) Use os restantes 25% dos dados para avaliação. Retorne a medida de RMSE.**

In [ ]:
rmse = 0

for n,row in servo_test.iterrows():
  if row[2] <= 3:
    if row[0] <= 1:
      new_class = folha1_val
    else:
      new_class = folha2_val
  else:
    if row[1] <= 1:
      new_class = folha3_val
    else:
      new_class = folha4_val
  
  rmse = rmse + np.power(row[4] - new_class, 2)

rmse = np.sqrt(rmse/servo_test.shape[0])
print("RMSE = {}".format(rmse))

RMSE = 0.6159676934967783


In [ ]:
#from sklearn.tree import DecisionTreeRegressor
#import matplotlib.pyplot as plt
#from sklearn import tree

#X = servo_train.iloc[:,:4]
#y = servo_train[4]

#regressor = DecisionTreeRegressor(max_depth=2, random_state=0)
#regressor.fit(X,y)

#x_test = servo_test.iloc[:,:4]
#y_pre = regressor.predict(x_test)

#tree.plot_tree(regressor, feature_names=columns_servo, filled=True)